# Phase 1 Notebook: `checkpoint_store.py`

Target file: `execution/langgraph/checkpoint_store.py`

Purpose: persist graph-state snapshots per node/step for resume/debug and audit trails.

## 3-Layer Context

- Layer 1 (Directive): durable execution and observability are core Phase 1 goals.
- Layer 2 (Orchestration): `graph.py` writes checkpoints after key node transitions.
- Layer 3 (Execution): SQLite checkpoint persistence is deterministic and queryable.

In [ ]:
from pathlib import Path
import sys

def bootstrap_repo_root() -> Path:
    cwd = Path.cwd().resolve()
    candidates = [cwd, *cwd.parents, Path('/home/nir/dev/agent_phase0')]
    for candidate in candidates:
        if (candidate / 'execution' / 'langgraph' / 'checkpoint_store.py').exists():
            if str(candidate) not in sys.path:
                sys.path.insert(0, str(candidate))
            return candidate
    raise RuntimeError('Could not locate repo root for checkpoint notebook.')

repo_root = bootstrap_repo_root()
print('repo_root =', repo_root)
print('kernel_python =', sys.executable)
if '/.venv/' not in sys.executable.replace('\\', '/'):
    print('WARNING: kernel is not the project .venv interpreter.')

## P0 vs P1 (Execution Traceability)

| Concern | Phase 0 | Phase 1 (`checkpoint_store.py`) | Why this matters |
|---|---|---|---|
| State persistence | transient in-memory loop | persistent node checkpoints | recoverability and post-mortem analysis |
| Replay support | logs only | latest-state load + checkpoint list | deterministic debugging path |
| Node visibility | implicit | `node_name` + `step` per checkpoint | better observability |

## Why this block exists: inspect persistence API

We inspect schema + save/load/list methods to verify replay and timeline capabilities.

In [ ]:
import inspect
from execution.langgraph.checkpoint_store import SQLiteCheckpointStore

print(inspect.getsource(SQLiteCheckpointStore._initialize_schema))
print(inspect.getsource(SQLiteCheckpointStore.save))
print(inspect.getsource(SQLiteCheckpointStore.load_latest))
print(inspect.getsource(SQLiteCheckpointStore.list_checkpoints))

## Why this block exists: checkpoint timeline demo

This demonstrates deterministic save-ordering and latest-state retrieval for a synthetic run.

In [ ]:
from execution.langgraph.checkpoint_store import SQLiteCheckpointStore
from execution.langgraph.state_schema import new_run_state

store = SQLiteCheckpointStore('.tmp/notebook_demo_checkpoints.db')
state = new_run_state('system prompt', 'user input', run_id='run-checkpoint-demo')
store.save(run_id='run-checkpoint-demo', step=0, node_name='init', state=state)
state['step'] = 1
state['final_answer'] = 'partial'
store.save(run_id='run-checkpoint-demo', step=1, node_name='plan', state=state)
timeline = store.list_checkpoints('run-checkpoint-demo')
latest = store.load_latest('run-checkpoint-demo')
timeline, latest['step'], latest['final_answer']

In [ ]:
assert len(timeline) >= 2
assert latest['step'] == 1
assert latest['final_answer'] == 'partial'
print('checkpoint store assertions passed')

## Takeaways

- Checkpoints are the Phase 1 durability bridge from raw loop to replayable execution.
- P1 adds node-level persistence that P0 did not model.
- This file supports future resume/HITL workflows directly.
- Next notebook: `execution/notebooks/p1_tools_registry.ipynb`.